In [29]:
import PyPDF2 
import textract

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [30]:
# Read Pdf file
files = ['Document1.pdf','Document2.pdf','Document3.pdf','Document4.pdf','Document5.pdf','Document6.pdf']

for filename in files:
    #open allows you to read the file
    pdfFileObj = open(filename,'rb')
    #The pdfReader variable is a readable object that will be parsed
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    #discerning the number of pages will allow us to parse through all #the pages
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    #The while loop will read each page
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()
    
    #This if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.
    if text != "":
       text = text
    #If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
    else:
       text = textract.process(fileurl, method='tesseract', language='eng')
    # Now we have a text variable which contains all the text derived #from our PDF file. Type print(text) to see what it contains. It #likely contains a lot of spaces, possibly junk such as '\n' etc.
    # Now, we will clean our text variable, and return it as a list of keywords.

In [31]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

True

In [32]:
# Convert text into keywords

#The word_tokenize() function will break our text phrases into #individual words
tokens = word_tokenize(text)
#we'll create a new list which contains punctuation we wish to clean
punctuations = ['(',')',';',':','[',']',',']
#We initialize the stopwords variable which is a list of words like #"The", "I", "and", etc. that don't hold much value as keywords
stop_words = stopwords.words('english')
#We create a list comprehension which only returns a list of words #that are NOT IN stop_words and NOT IN punctuations.
keywords = [word for word in tokens if not word in stop_words and not word in punctuations]

In [33]:
print keywords

In [34]:
keywords = filter(unicode.isalnum, keywords)
print keywords

In [35]:
#import csv
#import sys  
#reload(sys)  
#sys.setdefaultencoding('utf-8')

spam_list = ["Free", "Click here", "visit", "open attachment", "call this number", "money", "Out", "extra", "offer", 
 "available", "Pension", "Opportunity", "Chance", "Investment", "Pension"]

#with open('spam-output.csv', 'wb') as f:
#    writer = csv.writer(f)
#    for val in keywords:
#        if val in spam_list:
#            value = val + "," + "spam"
#            writer.writerow([value])
#       else:
#           value = val + "," + "notspam"
#           writer.writerow([value])

# Since most of the spam_or_not columns contains not spam, Manually adding spam keywords to the csv to train
keyword_spam_list = ["Free", "Click here", "visit", "open attachment", "call this number", "money", "Out", "extra", "offer", 
 "Chance", "Investment", "Pension"]

for each_value in keyword_spam_list:
    keywords.append(each_value)

print keywords

In [36]:
import csv
import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')

from itertools import izip_longest as zip_longest
spam_or_not = []
d = [keywords, spam_or_not]
export_data = zip_longest(*d, fillvalue = '')
for val in keywords:
    if val in spam_list:
        # 0 for spam
        spam_or_not.append("0")
    else:
        # 1 for not spam
        spam_or_not.append("1")
with open('spam-output.csv', 'wb') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(("keywords", "spam_or_not"))
    wr.writerows(export_data)
myfile.close()

In [37]:
#import re
#import pandas as pd
#dataset = pd.read_csv("spam-output.csv")
#corpus = []
#for i in range(len(keywords)):
#    keywords = re.sub('[^a-zA-Z]', '', dataset['keywords'][i])
#    corpus.append(keywords)

In [38]:
# Creating bag of words model
import pandas as pd
dataset = pd.read_csv("spam-output.csv")

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 150)
X = dataset.iloc[:, 0].values
X = cv.fit_transform(X).toarray()
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [39]:
y = dataset.iloc[:, 1].values
#y

In [40]:
# Applying naive bayes classifier

from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None)

In [41]:
y_pred = classifier.predict(X_test)